## Double check on pie chart numbers.

total: 583

feat_summary_varEp: 583

feat_summary (exclude all top10 univariate <= 0): 568

filtered (by score_test and corr_test_recall)

feat_summary_varEp: 529

feat_summary (exclude all top10 univariate <= 0): 526

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from ceres_infer.utils import getFeatGene, getFeatSource
from ceres_infer.analyses import generate_featSummary

In [3]:
df = pd.read_csv('../out/20.0216 feat/reg_rf_boruta/model_results.csv')

In [4]:
df.head(3)

,model,feature,target,score_train,score_test,score_oob,corr_test,corr_test_recall,score_p19q4,corr_p19q4,corr_p19q4_recall,feat_id,feat_idx
0,all,all,FGFR1,0.629550,0.102785,0.096981,0.344408,0.998,NaN,NaN,NaN,NaN,NaN
1,topfeat,topfeat,FGFR1,0.728297,0.401316,0.462546,0.510498,0.999,NaN,NaN,NaN,NaN,NaN
2,top10feat,top10feat,FGFR1,0.707737,0.432236,0.487608,0.403751,1.000,0.505877,0.300472,0.986,NaN,NaN


In [5]:
metric_eval = 'score_test'
def getVarExp(x):
    # get variance explained
    df = x.loc[x.model == 'univariate', ['feature', 'target', metric_eval]].copy()
    df.columns = ['feature', 'target', 'score_ind']
    df.score_ind = round(df.score_ind, 5)
    df['score_rd'] = round(x.loc[x.model == 'top10feat', metric_eval].values[0], 5) if sum(
        x.model == 'topfeat') > 0 else np.nan
    df['score_full'] = round(x.loc[x.model == 'all', metric_eval].values[0], 5)
    df['varExp_ofFull'] = round(df.score_ind / df.score_full, 5)
    df['varExp_ofRd'] = round(df.score_ind / df.score_rd, 5)
    df['feat_idx'] = list(range(1, df.shape[0] + 1))
    return df

varExp = df.groupby('target').apply(getVarExp)

In [6]:
varExp.reset_index(inplace=True, drop=True)
varExp['feat_gene'] = varExp['feature'].apply(getFeatGene, firstOnly=True)
varExp['feat_source'] = varExp['feature'].apply(getFeatSource, firstOnly=True)

In [7]:
varExp.target.nunique() # original list

583

In [19]:
varExp.loc[varExp.score_ind>0,:].target.nunique() # after filtering out target genes were R2s of all top 10 feat univariate models are negative
# this is done in feat_summary = generate_featSummary(varExp)

568

In [ ]:
# filtering step
# top 10 feat model with R2 > thresholds.score_rd10 (0.1)
# top 10 feat model with recall > thresholds.recall_rd10 (0.95)

df2 = df[df.model == 'top10feat']
df2_filtered = df2[(df2.score_test > 0.1) & (df2.corr_test_recall > 0.95)]
df2_filtered.target.nunique()

In [33]:
varExp_filtered = varExp[varExp.target.isin(df2_filtered.target)]
varExp_filtered.target.nunique()

529

In [34]:
varExp_filtered.loc[varExp_filtered.score_ind>0,:].target.nunique() 

526